In [1]:
import sys
import os
import onnx
import tensorflow as tf
import numpy as np
import tf2onnx

# from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense

2024-11-22 18:40:36.837569: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

In [3]:
model = Sequential([
    Flatten(input_shape=(28, 28)),  # Flatten 28x28 images to 1D vectors
    Dense(64, activation='relu'),  # Fully connected hidden layer
    Dense(10, activation='softmax')  # Output layer for 10 classes
])

2024-11-22 18:41:42.660911: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=3, batch_size=32)

Epoch 1/3
1875/1875 [==============================] - 11s 5ms/step - loss: 0.4539 - accuracy: 0.8400
Epoch 2/3
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3801 - accuracy: 0.8622
Epoch 3/3
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3464 - accuracy: 0.8744


In [6]:
predictions = model.predict(x_test)


313/313 [==============================] - 1s 3ms/step


In [7]:
sum(np.argmax(predictions, axis=1) == y_test) / len(y_test) * 100

85.7

Save model

In [8]:
tf.saved_model.save(model, "fashion_mnist_classification_model_tf_64")

INFO:tensorflow:Assets written to: fashion_mnist_classification_model_tf_64/assets


INFO:tensorflow:Assets written to: fashion_mnist_classification_model_tf_64/assets


In [10]:
!python -m tf2onnx.convert --saved-model fashion_mnist_classification_model_tf_64 --output fashion_mnist_classification_model_tf_64.onnx

/home/gatum/.conda/envs/marabou_3/lib/python3.10/runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2024-11-22 18:43:29,150 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2024-11-22 18:43:29,367 - INFO - Signatures found in model: [serving_default].
2024-11-22 18:43:29,367 - WARNING - '--signature_def' not specified, using first signature: serving_default
2024-11-22 18:43:29,367 - INFO - Output names: ['dense_1']
2024-11-22 18:43:29,456 - INFO - Using tensorflow=2.12.0, onnx=1.17.0, tf2onnx=1.16.1/15c810
2024-11-22 18:43:29,457 - INFO - Using opset <onnx, 15>
2024-11-22 18:43:29,462 - INFO - Computed 0 values for constant folding
2024-11-22 18:43:29,474 - INFO - Optimizing ONNX model
2024-11-22 18:43:29,509 - INFO - After optimization: Cast -1 (1->0), Identity -2 (2->0)
2024-11-22 18:43:29,510 - IN